In [1]:
import json
from typing import Sequence, List

from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.utils.function_calling import convert_to_openai_tool, convert_to_openai_function
from langchain_core.messages import HumanMessage




In [ ]:
model = "nexusraven"

In [35]:
llm = OllamaFunctions(
    model=model,
    format='json',
    context_window=8192
)

In [36]:
def add_two_numbers(number1: int, number2: int)-> int:
    """
    Add two numbers and return the result.
    
    Args:
        number1: The first number.
        number2: The second number.   
    """
    return (number1 + number2)

def multiply_two_numbers(number1: int, number2: int)-> int:
    """
    Multiply two numbers and return the result.
    
    Args:
        number1: The first number.
        number2: The second number.   
    """
    return (number1 * number2)


def useless_tool() -> int:
    """
    This is useless tool
    """
    return "This is a useless output"



In [37]:
tools = [convert_to_openai_tool(func)['function'] for func in [
    add_two_numbers, multiply_two_numbers, useless_tool
]]

In [38]:
tools[1]

{'name': 'multiply_two_numbers',
 'description': 'Multiply two numbers and return the result.',
 'parameters': {'type': 'object',
  'properties': {'number1': {'type': 'integer',
    'description': 'The first number.'},
   'number2': {'type': 'integer', 'description': 'The second number.'}},
  'required': ['number1', 'number2']}}

In [39]:
llm = llm.bind_tools(tools=tools)

In [40]:
llm.invoke("what is the sum of 3 and 4?")

AIMessage(content='', id='run-e1ab193e-3dd9-4241-bec2-eefc58760173-0', tool_calls=[{'name': 'add_two_numbers', 'args': {'number1': 3, 'number2': 4}, 'id': 'call_974d53bb4ee846dbbf35019d37329430'}])

In [41]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import BaseTool, FunctionTool



In [42]:
multiply_tool = FunctionTool.from_defaults(fn=multiply_two_numbers)
add_tool = FunctionTool.from_defaults(fn=add_two_numbers)


In [44]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model=model, request_timeout=60.0)

response = llm.complete("What is the capital of France?")
print(response)

The capital of France is Paris.


In [45]:
agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)

In [46]:
response = agent.chat("What is 5+3+2? Calculate step by step ")


Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add_two_numbers
Action Input: {'number1': 5, 'number2': 3}
Observation: 8
Thought: The calculation is not complete yet, I need to add another number to the result.
Action: add_two_numbers
Action Input: {'number1': 8, 'number2': 2}
Observation: 10
Thought: The final answer can be calculated now. I will use the user's language to answer.
Answer: The calculation step by step is 5+3=8 and then 8+2=10, so the final answer is 10.


In [47]:
response

AgentChatResponse(response='The calculation step by step is 5+3=8 and then 8+2=10, so the final answer is 10.', sources=[ToolOutput(content='8', tool_name='add_two_numbers', raw_input={'args': (), 'kwargs': {'number1': 5, 'number2': 3}}, raw_output=8, is_error=False), ToolOutput(content='10', tool_name='add_two_numbers', raw_input={'args': (), 'kwargs': {'number1': 8, 'number2': 2}}, raw_output=10, is_error=False)], source_nodes=[], is_dummy_stream=False)

In [48]:

prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do t